In [18]:
# - 売り上げ高
# - 営業利益
# まで
# かなりめんどくさい…

# todo 
# 営業活動キャッシュフロー
# 営業活動キャッシュフロー/売上=営業キャッシュフローマージン計算
# 営業キャッシュフローマージンをendDatetimeで降順ソートして直近5年(出せるとこまで、2年が限界か？)
# 算出し、15%以上が2年以上ならOperating_cash_flow_margin_listに銘柄を追加

In [19]:
import glob

from arelle.ModelValue import qname
from arelle import Cntlr
import numpy as np

import pandas as pd
pd.set_option('display.max_rows',1000)
import plotly.graph_objects as go

import stock_research as sr
taiouhyou = sr.taiouhyou


In [31]:
stock_code_list_path = '../warehouse/stock/created/submitted_doc_list_only_stock*.csv'
files = glob.glob(stock_code_list_path)
for file in files:
    print(file)

../warehouse/stock/created\submitted_doc_list_only_stock_20220330_20220428.csv


In [40]:
orig_data_path = '../warehouse/stock/original_data/'
stock_code_list = '../warehouse/stock/created/submitted_doc_list_only_stock_20220330_20220428.csv'
code_df = pd.read_csv(stock_code_list)

In [41]:
# code_listは銘柄コードごとにユニーク
tmp = code_df.groupby('filerName').size()
# tmp.name = 'count'
len(tmp.values)

132

In [43]:
def read_xbrl(file_path,stock_code) -> pd.DataFrame:
    ctrl = Cntlr.Cntlr(logFileName='logToPrint')
    modelXbrl = ctrl.modelManager.load(file_path)
    
    fact_df = pd.DataFrame(
    data=[(
        fact.concept.qname.localName, 
        fact.value,
        fact.isNumeric, 
        fact.contextID,
        fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None),
        fact.concept.label(preferredLabel=None, lang='en', linkroleHint=None),
        fact.context.startDatetime,
        fact.context.endDatetime,
        fact.context.instantDatetime,
        fact.decimals,
    ) for fact in modelXbrl.facts],
    columns=[
        "element_id", 
        "value",
        "isNumeric", 
        "contextID",
        "Ja_item_name",
        "en_item_name",     
        "startDatetime",
        "endDatetime",
        "instantDatetime",
        "decimals",
    ])
    return fact_df


In [44]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
j=-1
for stock_code in code_df['コード'].unique():
    j+=1
    if j==1:break
    df = []
    doc_id = code_df.query(f'コード=={stock_code}').docID.values[0]
    date = code_df.query(f'コード=={stock_code}').submitDateTime.values[0]
    company_name = code_df.query(f'コード=={stock_code}').filerName.values[0]

    file_path = orig_data_path + date[:10] + '/' + doc_id + '/XBRL/PublicDoc/*.xbrl'
    file_path = glob.glob(file_path)[0]
    
    fact_df = read_xbrl(file_path,stock_code)
    
    # プロットに向かない数値以外のデータをすべて除外
    mask = fact_df['value'].str.match('^[0-9]+$')
    fact_df = fact_df[mask]
    
    count = 0
    for k in taiouhyou:
        print(k)
        if k=="NetSalesSummaryOfBusinessResults":
            tmp = sr.get_sales(fact_df,k)
        elif k=="OperatingIncome":
            tmp = sr.get_OperatingIncome(fact_df,k)
        
        if count==0:
            df = tmp
            count+=1
            continue
        else:
            df = pd.concat([df,tmp],axis=0)
        count+=1
        
    # save_stock__statistics(df)

NetSalesSummaryOfBusinessResults
OperatingIncome


In [16]:
# def save_stock_statistics(df):
'''csvとして保存するための後処理'''
df = df.drop_duplicates()
min_date = df.endDatetime.min().strftime('%Y%m%d')
max_date = df.endDatetime.max().strftime('%Y%m%d')

for i,ele in enumerate(df.element_id.unique()):
    print(i)
    ele_df = df.query(f'element_id == "{ele}"')
    ele_df = ele_df.sort_values('endDatetime',ascending=False)
    YoY = sr.make_YoY(ele_df['value'])
    ele_df['YoY'] = YoY.values
    if i == 0:
        tmpdf = ele_df
    else:
        tmpdf = pd.concat([tmpdf,ele_df],axis=0)

# df.to_csv(f'../warehouse/stock/created/{stock_code}_statistics_{min_date}_{max_date}.csv')


0
1


In [ ]:
import datetime
thisyear = datetime.date.today().year
useyear_1 = thisyear-5
useyear_2 = thisyear+5
# ここに実績値をジョインしていけばいい。value_sales,value_operaing...とか
tmp = [str(i)+'-01'+'-01' for i in range(useyear_1,useyear_2)]
tmp = pd.DataFrame(tmp,columns=['endDatetime'])
tmp = pd.to_datetime(tmp["endDatetime"]) 

pd.DataFrame.merge(tmp,pd.concat([x,y],axis=1),how='left')
